In [1]:
import gsiberror as gb
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import panel as pn
import hvplot.xarray
import numpy as np
import intake

pn.extension()

ValueError: Missing SRI hash for version 3.3.4

In [ ]:
path = '/extra3/GitHub/GSIMonitor/new'
bcptec = 'berror_stats'
fcptec = os.path.join(path, bcptec)

cptec_b = gb.Berror(fcptec)
cptec_b.read_records()

In [ ]:
catalog_berror = intake.open_catalog('http://ftp1.cptec.inpe.br/pesquisa/das/carlos.bastarz/GSIMonitor/berror/catalog_berror.yml')

In [ ]:
catalog_berror['amplitudes_sf']

In [ ]:
level_lst = np.arange(0,64, 1).tolist()
level = pn.widgets.IntSlider(name='Level', value=0, start=level_lst[0], step=1, end=level_lst[-1])

balproj_lst = ['agvin', 'bgvin', 'wgvin']
balproj = pn.widgets.Select(name='Balance Projection Matrix', value=balproj_lst[0], options=balproj_lst)

stdevvars_lst = ['sf', 'vp', 't', 'q', 'qin', 'oz', 'ps', 'cw', 'sst']
stdevvars = pn.widgets.Select(name='Standard Deviation', value=stdevvars_lst[0], options=stdevvars_lst)

show_profile = pn.widgets.Checkbox(name='Profile', value=False)

hscalevars_lst = ['sf', 'vp', 't', 'q', 'oz', 'cw', 'ps', 'sst']
hscalevars = pn.widgets.Select(name='Horizontal Length Scale', value=hscalevars_lst[0], options=hscalevars_lst)

vscalevars_lst = ['sf', 'vp', 't', 'q', 'oz', 'cw']
vscalevars = pn.widgets.Select(name='Vertical Length Scale', value=vscalevars_lst[0], options=vscalevars_lst)

vertical_log = pn.widgets.Checkbox(name='Vertical Log', value=False)

In [ ]:
@pn.depends(balproj, level, vertical_log)
def plotBalProjs(balproj, level, vertical_log):
    dset = cptec_b.balprojs[balproj]
    if balproj == 'wgvin':
        ax = dset.isel(level=0, latitude=slice(0,-2)).hvplot.line(x='latitude', 
                                                                  title='Projection of the Stream Function over the balanced part of Surface Pressure')      
    elif balproj == 'bgvin':
        if vertical_log:
            ax = dset.hvplot.quadmesh(y='level', 
                                      x='latitude',
                                      logy=True,
                                      clabel='Km',
                                      aspect=1,
                                      cmap='jet',
                                      frame_height=500,
                                      title='Projection of the Stream Function over the balanced part of Velocity Potential')
        else:
            ax = dset.hvplot.quadmesh(y='level', 
                                      x='latitude',
                                      logy=False,
                                      clabel='Km',
                                      aspect=1,
                                      cmap='jet',
                                      frame_height=500,
                                      title='Projection of the Stream Function over the balanced part of Velocity Potential')            
    elif balproj == 'agvin':
        if vertical_log:
            ax = dset.isel(level_2=level).hvplot.quadmesh(y='level', 
                                                          x='latitude',
                                                          logy=True,
                                                          clabel='Km',
                                                          aspect=1,
                                                          cmap='jet',
                                                          frame_height=500,
                                                          title='Projection of Stream Function at level ' + str(level) + ' over the vertical \nprofile of the balanced part of Virtual Temperature')
        else:        
            ax = dset.isel(level_2=level).hvplot.quadmesh(y='level', 
                                                          x='latitude',
                                                          logy=False,
                                                          clabel='Km',
                                                          aspect=1,
                                                          cmap='jet',
                                                          frame_height=500,
                                                          title='Projection of Stream Function at level ' + str(level) + ' over the vertical \nprofile of the balanced part of Virtual Temperature')        
    return ax

In [ ]:
pn.Column(balproj, level, vertical_log, plotBalProjs).servable()

In [ ]:
@pn.depends(stdevvars, show_profile, vertical_log)
def plotStDev(stdevvars, show_profile, vertical_log):
    if stdevvars == 'sf': 
        vname = 'Stream Function'
    elif stdevvars == 'vp':
        vname = 'Velocity Potential'
    elif stdevvars == 't':
        vname = 'Unbalanced part of Temperature'
    elif stdevvars == 'q':
        vname = 'Relative Humidity'
    elif stdevvars == 'qin':
        vname = 'Relative Humidity'
    elif stdevvars == 'oz':
        vname = 'Ozone'
    elif stdevvars == 'cw':
        vname = 'Liquid Water Content'
    elif stdevvars == 'ps':
        vname = 'Surface Pressure'
    elif stdevvars == 'sst':
        vname = 'Sea Surface Temperature'

    if vertical_log:
        logy=True
    else:
        logy=False
    
    if stdevvars == 'qin':
        dset = cptec_b.amplitudes[stdevvars]*1e2
        if show_profile:
            ax = dset.isel(latitude=slice(0,25)).mean(dim='latitude').hvplot.line(#y='level',
                                                                 #x='latitude',
                                                                 clabel='Km',
                                                                 aspect=1,
                                                                 cmap='jet',
                                                                 frame_height=500,
                                                                 invert=True,
                                                                 logy=logy,
                                                                 title='Standard Deviation of ' + str(vname))
        else:
            ax = dset.isel(latitude=slice(0,25)).hvplot.quadmesh(y='level',
                                                                 x='latitude',
                                                                 clabel='Km',
                                                                 aspect=1,
                                                                 cmap='jet',
                                                                 frame_height=500,
                                                                 logy=logy,
                                                                 title='Standard Deviation of ' + str(vname))            
    elif stdevvars == 'ps':
        dset = cptec_b.amplitudes[stdevvars]
        ax = dset.hvplot.line(x='latitude',
                              clabel='Km',
                              aspect=1,
                              cmap='jet',
                              frame_height=500,
                              title='Standard Deviation of ' + str(vname))  
    elif stdevvars == 'sst':
        dset = cptec_b.amplitudes[stdevvars]
        ax = dset.hvplot.quadmesh(y='latitude',
                                  x='longitude',
                                  clabel='Km',
                                  aspect=1,
                                  cmap='jet',
                                  frame_height=500,
                                  geo=True,
                                  coastline=True,  
                                  title='Standard Deviation of ' + str(vname))          
    else:
        dset = cptec_b.amplitudes[stdevvars]
        if show_profile:
            ax = dset.mean(dim='latitude').hvplot.line(#y='level',
                                      #x='latitude',
                                      clabel='Km',
                                      aspect=1,
                                      cmap='jet',
                                      frame_height=500,
                                      invert=True,
                                      #show_grid=True,
                                      logy=logy,
                                      title='Standard Deviation of ' + str(vname))        
        else:
            ax = dset.hvplot.quadmesh(y='level',
                                      x='latitude',
                                      clabel='Km',
                                      aspect=1,
                                      cmap='jet',
                                      frame_height=500, 
                                      logy=logy,
                                      title='Standard Deviation of ' + str(vname))         
    return ax

In [ ]:
pn.Column(stdevvars, show_profile, vertical_log, plotStDev).servable()

In [ ]:
@pn.depends(hscalevars, vertical_log)
def plotHScale(hscalevars, vertical_log):
    if hscalevars == 'sf': 
        vname = 'Stream Function'
    elif hscalevars == 'vp':
        vname = 'Velocity Potential'
    elif hscalevars == 't':
        vname = 'Unbalanced part of Temperature'
    elif hscalevars == 'q':
        vname = 'Relative Humidity'
    elif hscalevars == 'qin':
        vname = 'Relative Humidity'
    elif hscalevars == 'oz':
        vname = 'Ozone'
    elif hscalevars == 'cw':
        vname = 'Liquid Water Content'
    elif hscalevars == 'ps':
        vname = 'Surface Pressure'
    elif hscalevars == 'sst':
        vname = 'Sea Surface Temperature'

    if vertical_log:
        logy=True
    else:
        logy=False
    
    if hscalevars == 'qin':
        dset = cptec_b.hscales[hscalevars]*1e2
        ax = dset.isel(latitude=slice(0,25)).hvplot.quadmesh(y='level',
                                                             x='latitude',
                                                             clabel='Km',
                                                             aspect=1,
                                                             cmap='jet',
                                                             frame_height=500,
                                                             logy=logy,
                                                             title='Horizontal Length Scale of ' + str(vname))
    elif hscalevars == 'ps':
        dset = cptec_b.hscales[hscalevars]
        ax = dset.hvplot.line(x='latitude',
                              clabel='Km',
                              aspect=1,
                              cmap='jet',
                              frame_height=500,
                              title='Horizontal Length Scale of ' + str(vname))  
    elif hscalevars == 'sst':
        dset = cptec_b.hscales[hscalevars]
        ax = dset.hvplot.quadmesh(y='latitude',
                                  x='longitude',
                                  clabel='Km',
                                  aspect=1,
                                  cmap='jet',
                                  frame_height=500,
                                  geo=True,
                                  coastline=True,  
                                  title='Horizontal Length Scale of ' + str(vname))          
    else:
        dset = cptec_b.hscales[hscalevars]
        ax = dset.hvplot.quadmesh(y='level',
                                  x='latitude',
                                  clabel='Km',
                                  aspect=1,
                                  cmap='jet',
                                  frame_height=500, 
                                  logy=logy,
                                  title='Horizontal Length Scale of ' + str(vname))        
    return ax

In [ ]:
pn.Column(hscalevars, vertical_log, plotHScale).servable()

In [ ]:
@pn.depends(vscalevars, vertical_log)
def plotVScale(vscalevars, vertical_log):
    if vscalevars == 'sf': 
        vname = 'Stream Function'
    elif vscalevars == 'vp':
        vname = 'Velocity Potential'
    elif vscalevars == 't':
        vname = 'Unbalanced part of Temperature'
    elif vscalevars == 'q':
        vname = 'Relative Humidity'
    elif vscalevars == 'qin':
        vname = 'Relative Humidity'
    elif vscalevars == 'oz':
        vname = 'Ozone'
    elif vscalevars == 'cw':
        vname = 'Liquid Water Content'
    elif vscalevars == 'ps':
        vname = 'Surface Pressure'
    elif vscalevars == 'sst':
        vname = 'Sea Surface Temperature'

    if vertical_log:
        logy=True
    else:
        logy=False
    
    if vscalevars == 'qin':
        dset = cptec_b.vscales[vscalevars]*1e2
        ax = dset.isel(latitude=slice(0,25)).hvplot.quadmesh(y='level',
                                                             x='latitude',
                                                             clabel='Km',
                                                             aspect=1,
                                                             cmap='jet',
                                                             frame_height=500,
                                                             logy=logy,
                                                             title='Vertical Length Scale of ' + str(vname))
    elif vscalevars == 'ps':
        dset = cptec_b.vscales[vscalevars]
        ax = dset.hvplot.line(x='latitude',
                              clabel='Km',
                              aspect=1,
                              cmap='jet',
                              frame_height=500,
                              title='Vertical Length Scale of ' + str(vname))  
    elif vscalevars == 'sst':
        dset = cptec_b.vscales[vscalevars]
        ax = dset.hvplot.quadmesh(y='latitude',
                                  x='longitude',
                                  clabel='Km',
                                  aspect=1,
                                  cmap='jet',
                                  frame_height=500,
                                  geo=True,
                                  coastline=True,  
                                  title='Vertical Length Scale of ' + str(vname))          
    else:
        dset = cptec_b.vscales[vscalevars]
        ax = dset.hvplot.quadmesh(y='level',
                                  x='latitude',
                                  clabel='Km',
                                  aspect=1,
                                  cmap='jet',
                                  frame_height=500, 
                                  logy=logy,
                                  title='Vertical Length Scale of ' + str(vname))        
    return ax

In [ ]:
pn.Column(vscalevars, vertical_log, plotVScale).servable()

In [ ]:
for var in balproj_lst:
    cptec_b.balprojs[var].to_zarr('smna_berror_balprojs_' + str(var) + '.zarr', mode='w', consolidated=True)

In [ ]:
for var in stdevvars_lst:
    cptec_b.amplitudes[var].to_zarr('smna_berror_amplitudes_' + str(var) + '.zarr', mode='w', consolidated=True)

In [ ]:
for var in hscalevars_lst:
    cptec_b.hscales[var].to_zarr('smna_berror_hscales_' + str(var) + '.zarr', mode='w', consolidated=True)

In [ ]:
for var in vscalevars_lst:
    cptec_b.vscales[var].to_zarr('smna_berror_vscales_' + str(var) + '.zarr', mode='w', consolidated=True)